## Processin SIM data

In [7]:
from pysus.online_data.SIM import download
from pysus.preprocessing.decoders import translate_variables_SIM
from pysus.preprocessing.SIM import group_and_count, redistribute

In [8]:
df = download('sp',2010)
df

,CONTADOR,ORIGEM,TIPOBITO,DTOBITO,HORAOBITO,NATURAL,DTNASC,IDADE,SEXO,RACACOR,...,DTCADASTRO,ATESTANTE,FONTEINV,DTRECEBIM,UFINFORM,CB_PRE,MORTEPARTO,DTCADINF,TPOBITOCOR,DTCADINV
0,1,1,2,22072010,0700,829,18011933,477,1,1,...,28072010,1,,30082010,,J180,,,,
1,2,1,2,30082010,0700,831,05111945,464,2,2,...,14092010,1,,13102010,,C169,,,,
2,3,1,2,25092010,1700,835,26051938,472,1,3,...,13102010,1,,27102010,,I219,,,,
3,4,1,2,29012010,0740,835,04031966,443,1,,...,22022010,1,,12032010,,B208,,,,
4,5,1,2,01072010,1600,831,04111927,482,1,1,...,10082010,2,,12082010,,I251,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264946,264947,3,2,25122010,0345,826,28031930,480,1,,...,,4,,,,,,,,
264947,264948,3,2,25122010,1800,835,19111945,465,1,,...,,,,,,,,,,
264948,264949,3,2,25122010,2100,835,27071927,483,2,,...,,,,,,,,,,
264949,264950,3,2,25122010,2005,835,22061909,501,2,,...,,,,,,,,,,


In [9]:
variables = ['CODMUNRES','SEXO','IDADE_ANOS','CID10_CHAPTER']
df = translate_variables_SIM(df,age_classes=True,classify_cid10_chapters=True)
df = df[variables]
df

,CODMUNRES,SEXO,IDADE_ANOS,CID10_CHAPTER
0,nan,Masculino,"[77.0, 78.0)",10
1,nan,Feminino,"[64.0, 65.0)",2
2,nan,Masculino,"[72.0, 73.0)",9
3,nan,Masculino,"[43.0, 44.0)",1
4,nan,Masculino,"[82.0, 83.0)",9
...,...,...,...,...
264946,nan,Masculino,"[80.0, 81.0)",9
264947,nan,Masculino,"[65.0, 66.0)",2
264948,nan,Feminino,"[83.0, 84.0)",5
264949,nan,Feminino,"[90.0, inf)",9


In [10]:
counts = group_and_count(df,variables)
counts

,CODMUNRES,SEXO,IDADE_ANOS,CID10_CHAPTER,CONTAGEM
0,nan,Feminino,"[0.0, 1.0)",1,122.0
1,nan,Feminino,"[0.0, 1.0)",2,8.0
2,nan,Feminino,"[0.0, 1.0)",3,8.0
3,nan,Feminino,"[0.0, 1.0)",4,31.0
4,nan,Feminino,"[0.0, 1.0)",5,0.0
...,...,...,...,...,...
5239,nan,nan,nan,15,0.0
5240,nan,nan,nan,16,0.0
5241,nan,nan,nan,17,0.0
5242,nan,nan,nan,18,12.0


In [6]:
counts = redistribute(counts,['CODMUNRES','SEXO','IDADE_ANOS'])
counts

KeyboardInterrupt: 